In [ ]:
##############################################
# Task subdivision based on reachability     #
##############################################

# Packages
import numpy as np
import rospy
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import HybridSegment
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import DMPLWRhardcoded
from core_robotics.PyBSpline import BSplineSurface
from scipy.spatial.transform import Rotation as ScipyR
from scipy.spatial.transform import Slerp

In [ ]:
##############################################
# Helper Functions                           #
##############################################


# starting vals is a list of starting vals
# ending vals is a list of ending vals
# num_pts is how many interp values
# quat_vars is the index of any starts of quaternions so they can be SLERPed instead of LERPed
# super_pos_vars is the starting index for 2-pairs of variables which should have an added sine wave (e.g., rubbing)
# super_pos_freq is the frequency for the starting index pairs
# super_pos_amp is the amplitude for the starting index pairs
def interpMultD(starting_vals,ending_vals,num_pts,quat_vars=[], super_pos_vars = [], super_pos_freq=[], super_pos_amp = []):
    vals = []
    
    # LERP ALL VARIABLES
    for ii in range(0,num_pts):
       
        # super is to potentially add a circular motion on top of the LERP motion
        freq_super = 0.0
        amp_super = 0.0
        added_sine = np.zeros((np.shape(starting_vals)))
        for jj in range(0,len(super_pos_vars)):
            amp = super_pos_amp[jj]
            freq = super_pos_freq[jj]
            added_sine[super_pos_vars[jj]] = np.sin(np.pi*float(ii)/num_pts)*amp*np.sin(freq*float(ii)/2*np.pi)
            added_sine[super_pos_vars[jj]+1] = np.sin(np.pi*float(ii)/num_pts)*amp*np.cos(freq*float(ii)/2*np.pi)
        c_i = float(ii)/float(num_pts) # interpolation coefficient
        vals.append(list(c_i*(e-s)+s+a for s,e,a in zip(starting_vals,ending_vals,added_sine)))
    
    vals = np.asarray(vals) # currently,num_samples x num_vars
    
    # SLERP QUATERIONS
    for jj in range(0,len(quat_vars)):
        starting_q = np.array(starting_vals[quat_vars[jj]:quat_vars[jj]+4])
        ending_q = np.array(ending_vals[quat_vars[jj]:quat_vars[jj]+4])
        key_times = [0,num_pts-1]
        key_rots = ScipyR.from_quat(np.concatenate((starting_q.reshape((1,4)),ending_q.reshape((1,4))),axis=0))
        slerper = Slerp(key_times, key_rots)
        
        pts = list(range(0,num_pts))
        slerped = slerper(pts)
        vals[:,quat_vars[jj]:quat_vars[jj]+4] = np.asarray(slerped.as_quat())
    
    return vals.T # num vars x num_samples


In [ ]:
##############################################
# Create data for testing                    #
##############################################
rospack = rospkg.RosPack()
uliconfigpath = rospack.get_path('uli_config')


t_surf = np.array([0.43070987363200586, -0.013269419259123479, 0.06385816493811966])
R_surf = np.array([0.008339958880240922, 0.004026424369577458, -0.029073916288389654, 0.9995343617826891])
surf_file = uliconfigpath + 'registration_models/something.something'


starting = [[0.1, 0.1], [0.3, 0.1]]
ending = [[0.1, 0.9], [0.3, 0.9]]
samps = [100,100]

trajectories = []

for ii in range(0,len(starting)):
    temp_vars = interpMultD(starting[ii],ending[ii],samps[ii])
    data_temp = np.zeros((7,samps[ii]))
    
    for jj in range(0,np.shape(temp_vars)[1]):
        # get surface coordinates and apply surface transform
    
    trajectories.append(data_temp)


In [ ]:
##############################################
# Run Splitting algorithm                    #
##############################################

# input to algorithm is a list of 7xsamps trajectories and a urdf
urdf_file = ''